<a href="https://colab.research.google.com/github/baeseungyou/study/blob/main/9%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.9 MB/s eta 0:00:00


In [8]:
import re
import os
import requests
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, LSTM, Dense, Masking
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from konlpy.tag import Okt

In [9]:
url = 'http://www.manythings.org/anki/kor-eng.zip'
filename = 'kor-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)
response.raise_for_status()

with open(zipfilename, 'wb') as out_file:
    out_file.write(response.content)

try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
        zip_ref.extractall(path)
    print("압축 해제 완료!")
except zipfile.BadZipFile:
    print("파일이 손상되었습니다. 다운로드를 다시 확인해 주세요.")

압축 해제 완료!


In [10]:
okt = Okt()

In [11]:
def preprocess_english(text):
    text = text.lower()
    text = re.sub(r"([?.!,¿])", r" \1", text)
    text = re.sub(r"[^a-zA-Z0-9!.?]+", " ", text)

    return text

def preprocess_korean(text):
    text = re.sub(r"([?.!,¿])", r" \1", text)
    text = re.sub(r"[^ㄱ-ㅎ가-힣0-9!.?]+", " ", text)
    text = okt.morphs(text)

    return " ".join(text)

In [12]:
num_samples = 33000

In [15]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("kor.txt", "r") as lines:
    for i, line in enumerate(lines):

      src_line, tar_line, _ = line.strip().split('\t')

      src_line = [w for w in preprocess_english(src_line).split()]

      tar_line = preprocess_korean(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [16]:
en_sent = u"Have you had dinner?"
kr_sent = u"저녁은 드셨나요?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_english(en_sent))
print('전처리 전 한국어 문장 :', kr_sent)
print('전처리 후 한국어 문장 :', preprocess_korean(kr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 한국어 문장 : 저녁은 드셨나요?
전처리 후 한국어 문장 : 저녁 은 드셨나요 ?


In [17]:
sents_en_in, sents_kor_in, sents_kor_out = load_preprocessed_data()

In [18]:
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_kor_in[:5])
print('디코더의 레이블 :',sents_kor_out[:5])

인코더의 입력 : [['go', '.'], ['hi', '.'], ['run', '!'], ['run', '.'], ['who', '?']]
디코더의 입력 : [['<sos>', '가', '.'], ['<sos>', '안녕', '.'], ['<sos>', '뛰어', '!'], ['<sos>', '뛰어', '.'], ['<sos>', '누구', '?']]
디코더의 레이블 : [['가', '.', '<eos>'], ['안녕', '.', '<eos>'], ['뛰어', '!', '<eos>'], ['뛰어', '.', '<eos>'], ['누구', '?', '<eos>']]


In [19]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_kor = Tokenizer(filters="", lower=False)
tokenizer_kor.fit_on_texts(sents_kor_in)
tokenizer_kor.fit_on_texts(sents_kor_out)

decoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_kor.texts_to_sequences(sents_kor_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [20]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (5890, 102)
디코더의 입력의 크기(shape) : (5890, 127)
디코더의 레이블의 크기(shape) : (5890, 127)


In [21]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_kor.word_index
index_to_tar = tokenizer_kor.index_word

In [22]:
max_src_len = encoder_input.shape[1]
max_tar_len = decoder_input.shape[1]
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 102
target 문장의 최대 길이 : 127


In [23]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_kor.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 한국어어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 3141, 한국어어 단어 집합의 크기 : 5617


In [24]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [4908 3190 3386 ... 1727 4869 2295]


In [25]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [26]:
len(encoder_input)

5890

In [27]:
encoder_input[5000]

array([ 60, 164, 278,  23,  37,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [28]:
decoder_input[5000]

array([   2,   14,   23,    6,   38,   12, 1344,   29,    1,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [29]:
n_of_val = int(5890*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 589


In [30]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [31]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (5301, 102)
훈련 target 데이터의 크기 : (5301, 127)
훈련 target 레이블의 크기 : (5301, 127)
테스트 source 데이터의 크기 : (589, 102)
테스트 target 데이터의 크기 : (589, 127)
테스트 target 레이블의 크기 : (589, 127)


In [32]:
embedding_dim = 128
hidden_units = 128

In [33]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [34]:
decoder_inputs = Input(shape=(None))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [35]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [36]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [37]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, None, 128)      │        402,048 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None, 128)      │              0 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, None, 128)      │        718,976 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking (Masking)         │ (None, None, 128)      │              0 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ any (Any)                 │ (None, None)           │              0 │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking_1 (Masking)       │ (None, None, 128)      │              0 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ [(None, 128), (None,   │        131,584 │ masking[0][0],         │
│                           │ 128), (None, 128)]     │                │ any[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ [(None, None, 128),    │        131,584 │ masking_1[0][0],       │
│                           │ (None, 128), (None,    │                │ lstm_6[0][1],          │
│                           │ 128)]                  │                │ lstm_6[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, None, 5617)     │        724,593 │ lstm_7[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,108,785 (8.04 MB)

 Trainable params: 2,108,785 (8.04 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=200)

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - acc: 0.8409 - loss: 5.8147 - val_acc: 0.9349 - val_loss: 0.5528
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - acc: 0.9363 - loss: 0.4857 - val_acc: 0.9349 - val_loss: 0.4210
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - acc: 0.9364 - loss: 0.3979 - val_acc: 0.9351 - val_loss: 0.3896
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - acc: 0.9380 - loss: 0.3737 - val_acc: 0.9445 - val_loss: 0.3719
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - acc: 0.9462 - loss: 0.3576 - val_acc: 0.9482 - val_loss: 0.3597
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - acc: 0.9510 - loss: 0.3449 - val_acc: 0.9513 - val_loss: 0.3500
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - acc: 0.9526 - loss: 0.3372 - val_acc: 0.9514 - val_loss: 0.3426
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - acc: 0.9528 - loss: 0.3282 - val_acc: 0.9515 - val_loss: 0.3362
Epoch 9/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - a

In [39]:
encoder_model = Model(encoder_inputs, encoder_states)

In [40]:
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

In [41]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [42]:
def decode_sequence(input_seq):

  states_value = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''


  while not stop_condition:

    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]


    decoded_sentence += ' '+sampled_char

    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index


    states_value = [h, c]

  return decoded_sentence

In [43]:
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word!=0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word!=0 and encoded_word!=tar_to_index['<sos>'] and encoded_word!=tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [44]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
입력문장 : she suffers from low blood pressure . 
정답문장 : 그 사람 은 저혈압 에 시달리고 있어 . 
번역문장 : 그 사람 은 미혼 이야 . 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
입력문장 : i later regretted that decision . 
정답문장 : 나 는 그 결정 을 나중 에서야 후회 했어 . 
번역문장 : 나 는 그 의 장례식 에 참석 했다 . 
-------------------

In [45]:
for seq_index in [3,50,100,300,500]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
입력문장 : i think that you re wrong . 
정답문장 : 나 는 네 가 잘 못 했다고 생각 한다 . 
번역문장 : 난 네 가 부럽지 않아 . 
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━